# Load Dataset

In [1]:
import pandas as pd

In [2]:
df_radish = pd.read_excel('배추 가격(2014.1.3~2020.12.31).xlsx')
df_cabbage = pd.read_excel('무 가격(2014.1.3~2020.12.31).xlsx')
df_weather = pd.read_csv('기후데이터2.csv', encoding='cp949')

# Data Preprocessing

In [3]:
# 필요없는 column 삭제
df_cabbage_new = df_cabbage.drop(['품목', '법인', '거래량(톤)', '거래금액(백만원)', '광역산지'], axis=1)

# 가락시장에서 거래되는 무의 품종 중 가을무, 고랭지무, 월동무만 추출, 시군산지가 미분류로 결측치인 row 제거
df_cabbage_new = df_cabbage_new[((df_cabbage_new['품종'].isin(['가을무', '고냉지무', '월동무']))\
                                 & (df_cabbage_new['시장']=='서울가락도매') & (df_cabbage_new['시군산지']!='미분류'))]

In [4]:
# 필요없는 column 삭제
df_radish_new = df_radish.drop(['품목', '법인', '거래량(톤)', '거래금액(백만원)', '광역산지'], axis=1)

# 가락시장에서 거래되는 배추의 품종 중 김장(가을)배추, 고랭지배추, 월동배추만 추출, 시군산지가 미분류로 결측치인 row 제거
df_radish_new = df_radish_new[((df_radish_new['품종'].isin(['김장(가을)배추', '고냉지배추', '월동배추']))\
                                 & (df_radish_new['시장']=='서울가락도매') & (df_radish_new['시군산지']!='미분류'))]

In [5]:
# 기상 데이터와 배추, 무의 지역이 같은 것만 추출
weather_spot_value = df_weather['지점명'].unique()
cabbage_spot_value = df_cabbage_new['시군산지'].unique()
radish_spot_value = df_radish_new['시군산지'].unique()

cabbage_and_weather_spot = [x for x in weather_spot_value if x in cabbage_spot_value]
radish_and_weather_spot = [x for x in weather_spot_value if x in radish_spot_value]

In [6]:
# 필요없는 column 제거, 공통되는 지역만 추출
df_weather_cabbage = df_weather[df_weather['지점명'].isin(cabbage_and_weather_spot)]
df_weather_cabbage_new = df_weather_cabbage.drop(['지점'], axis=1)

df_weather_radish = df_weather[df_weather['지점명'].isin(radish_and_weather_spot)]
df_weather_radish_new = df_weather_radish.drop(['지점'], axis=1)

In [7]:
# 데이터 병합을 위해 임시로 만든 데이터프레임
df_for_merge_cabbage = pd.DataFrame({'지점명' : ['for_merge'],
                                    '일시' : ['2020-12-30'],
                                    '평균기온(°C)' : [0],
                                    '최저기온(°C)' : [0],
                                    '최고기온(°C)' : [0],
                                    '일강수량(mm)' : [0],
                                    '최대 풍속(m/s)' : [0],
                                    '평균 풍속(m/s)' : [0],
                                    '평균 상대습도(%)' : [0],
                                    '합계 일조시간(hr)' : [0],
                                    '합계 일사량(MJ/m2)' : [0],
                                    '일 최심적설(cm)' : [0]
                                    })

df_for_merge_radish = pd.DataFrame({'지점명' : ['for_merge'],
                                    '일시' : ['2020-12-30'],
                                    '평균기온(°C)' : [0],
                                    '최저기온(°C)' : [0],
                                    '최고기온(°C)' : [0],
                                    '일강수량(mm)' : [0],
                                    '최대 풍속(m/s)' : [0],
                                    '평균 풍속(m/s)' : [0],
                                    '평균 상대습도(%)' : [0],
                                    '합계 일조시간(hr)' : [0],
                                    '합계 일사량(MJ/m2)' : [0],
                                    '일 최심적설(cm)' : [0]
                                    })

In [8]:
# 기간, 지역이 같은 것에 대한 기상 데이터 병합
for cabbage_date, cabbage_spot in zip(df_cabbage_new['일자'], df_cabbage_new['시군산지']):
    inner_data_cabbage = df_weather_cabbage_new[((df_weather_cabbage_new['일시']==cabbage_date.strftime("%Y-%m-%d")) \
                                                 & (df_weather_cabbage_new['지점명']==cabbage_spot))]
    df_for_merge_cabbage = pd.concat([df_for_merge_cabbage, inner_data_cabbage])

In [9]:
# 기간, 지역이 같은 것에 대한 기상 데이터 병합
for radish_time, radish_spot in zip(df_radish_new['일자'], df_radish_new['시군산지']):
    inner_data_radish = df_weather_radish_new[((df_weather_radish_new['일시']==radish_time.strftime("%Y-%m-%d")) \
                                               & (df_weather_radish_new['지점명']==radish_spot))]
    df_for_merge_radish = pd.concat([df_for_merge_radish, inner_data_radish])

In [10]:
# 임시로 만들었던 row 제거
df_for_merge_cabbage_new = df_for_merge_cabbage.drop([0])
df_for_merge_radish_new = df_for_merge_radish.drop([0])

In [11]:
# merge를 위해 기준으로 잡을 column명 통일
df_for_merge_cabbage_new = df_for_merge_cabbage_new.rename(columns={'지점명':'시군산지',
                                                                    '일시':'일자'})

df_for_merge_radish_new = df_for_merge_radish_new.rename(columns={'지점명':'시군산지',
                                                                  '일시':'일자'})

In [12]:
# 중복 값 제거, index 초기화
df_for_merge_cabbage_new.drop_duplicates(['시군산지', '일자'], ignore_index=True, inplace=True)
df_for_merge_radish_new.drop_duplicates(['시군산지', '일자'], ignore_index=True, inplace=True)

In [13]:
# 병합을 위해 같은 형태로 맞춰여함. datetime -> object로 변환
def datetimeTostr(df):
    return df.strftime("%Y-%m-%d")

df_cabbage_new['일자'] = df_cabbage_new['일자'].apply(datetimeTostr)
df_radish_new['일자'] = df_radish_new['일자'].apply(datetimeTostr)

# Create New Dataset

In [14]:
# inner로 공통되는 부분끼리 합침
merge_inner_cabbage = pd.merge(df_cabbage_new, df_for_merge_cabbage_new, how='inner',on=['일자', '시군산지'])
merge_inner_radish = pd.merge(df_radish_new, df_for_merge_radish_new, how='inner',on=['일자', '시군산지'])

In [15]:
merge_inner_cabbage

,일자,품종,시장,시군산지,도매가격(원/kg),평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),최대 풍속(m/s),평균 풍속(m/s),평균 상대습도(%),합계 일조시간(hr),합계 일사량(MJ/m2),일 최심적설(cm)
0,2020-12-30,가을무,서울가락도매,고창,498.184624,-5.5,-8.9,-1.2,7.9,10.6,6.5,81.8,1.3,6.97,NaN
1,2020-12-30,가을무,서울가락도매,부안,605.009960,-6.0,-8.4,-0.6,19.3,5.2,2.1,89.9,0.0,NaN,NaN
2,2020-12-30,가을무,서울가락도매,해남,280.142405,-3.7,-5.0,-0.5,0.6,8.7,3.7,76.0,2.2,NaN,NaN
3,2020-12-30,가을무,서울가락도매,춘천,352.125749,-9.5,-13.6,-2.4,NaN,4.8,2.7,41.8,7.8,10.04,NaN
4,2020-12-29,가을무,서울가락도매,고창,443.141298,1.9,-1.7,7.4,2.6,8.1,3.3,89.8,0.6,4.19,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3745,2014-01-08,월동무,서울가락도매,제주,253.391990,7.0,4.2,10.7,41.3,8.6,5.8,77.8,0.0,0.56,NaN
3746,2014-01-07,월동무,서울가락도매,제주,252.749721,8.8,3.4,12.3,0.1,5.0,2.3,63.1,4.0,7.44,NaN
3747,2014-01-06,월동무,서울가락도매,제주,269.861962,6.3,2.5,10.6,NaN,4.6,2.3,51.9,8.8,10.52,NaN
3748,2014-01-04,월동무,서울가락도매,제주,315.257460,7.6,5.0,11.0,NaN,5.0,3.4,61.6,7.2,10.68,NaN


In [16]:
merge_inner_radish

,일자,품종,시장,시군산지,도매가격(원/kg),평균기온(°C),최저기온(°C),최고기온(°C),일강수량(mm),최대 풍속(m/s),평균 풍속(m/s),평균 상대습도(%),합계 일조시간(hr),합계 일사량(MJ/m2),일 최심적설(cm)
0,2020-12-30,김장(가을)배추,서울가락도매,해남,275.620438,-3.7,-5.0,-0.5,0.6,8.7,3.7,76.0,2.2,NaN,NaN
1,2020-12-30,김장(가을)배추,서울가락도매,해남,276.000000,-3.7,-5.0,-0.5,0.6,8.7,3.7,76.0,2.2,NaN,NaN
2,2020-12-29,김장(가을)배추,서울가락도매,해남,318.068019,2.4,-1.2,11.6,4.4,8.5,2.5,88.5,1.9,NaN,NaN
3,2020-12-29,김장(가을)배추,서울가락도매,해남,276.426600,2.4,-1.2,11.6,4.4,8.5,2.5,88.5,1.9,NaN,NaN
4,2020-12-28,김장(가을)배추,서울가락도매,해남,307.441098,4.0,-0.7,13.7,NaN,3.4,0.6,93.1,6.4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6982,2014-01-04,월동배추,서울가락도매,해남,318.022099,3.3,-0.6,7.0,NaN,5.3,2.9,60.4,9.3,NaN,NaN
6983,2014-01-04,월동배추,서울가락도매,해남,450.800000,3.3,-0.6,7.0,NaN,5.3,2.9,60.4,9.3,NaN,NaN
6984,2014-01-03,월동배추,서울가락도매,해남,415.073395,4.9,-3.8,13.5,NaN,3.2,0.9,59.3,5.7,NaN,NaN
6985,2014-01-03,월동배추,서울가락도매,해남,416.986784,4.9,-3.8,13.5,NaN,3.2,0.9,59.3,5.7,NaN,NaN


In [17]:
# csv로 추출
merge_inner_cabbage.to_csv('final_cabbage_and_weather.csv')
merge_inner_radish.to_csv('final_radish_and_weather.csv')